In [1]:
!pip install pandas openpyxl

In [2]:
import pandas as pd
import random

# Leer el archivo Excel con varias hojas
archivo = 'ventas_olap.xlsx'
excel = pd.read_excel(archivo, sheet_name=None)  # Lee todas las hojas

# Acceder a la hoja de Ventas
ventas_df = excel['Ventas']

# Opciones de regiones
regiones = ['norte', 'sur', 'este', 'oeste']

# Generar regiones aleatorias
ventas_df['region'] = [random.choice(regiones) for _ in range(len(ventas_df))]

# Sobrescribir la hoja de Ventas en el mismo archivo
excel['Ventas'] = ventas_df

# Guardar el archivo con las modificaciones
with pd.ExcelWriter(archivo, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    for hoja, df in excel.items():
        df.to_excel(writer, sheet_name=hoja, index=False)


In [3]:
import pandas as pd

# Ruta del archivo Excel original
archivo = 'ventas_olap.xlsx'

# Leer todas las hojas del archivo
excel = pd.read_excel(archivo, sheet_name=None)

# Acceder a la hoja 'Ventas'
ventas_df = excel['Ventas']

# Asegurar que la columna 'fecha' es tipo datetime
ventas_df['fecha'] = pd.to_datetime(ventas_df['fecha'])

# Crear el nuevo DataFrame para la hoja 'Fechas'
fechas_df = pd.DataFrame({
    'id_venta': ventas_df['id_venta'],
    'id_cliente': ventas_df['id_cliente'],
    'mes': ventas_df['fecha'].dt.month,
    'trimestre': ventas_df['fecha'].dt.quarter,
    'año': ventas_df['fecha'].dt.year
})

# Agregar la nueva hoja al diccionario
excel['Fechas'] = fechas_df

# Guardar el archivo con la nueva hoja
with pd.ExcelWriter(archivo, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    for hoja, df in excel.items():
        df.to_excel(writer, sheet_name=hoja, index=False)


In [6]:
import pandas as pd
import random

# Leer el archivo Excel
archivo = 'ventas_olap.xlsx'
excel = pd.read_excel(archivo, sheet_name=None)

# Obtener clientes únicos desde la hoja 'Ventas'
ventas_df = excel['Ventas']
clientes_unicos = ventas_df['id_cliente'].drop_duplicates().reset_index(drop=True)

# Lista de 20 ciudades mexicanas sin acentos y de una sola palabra
ciudades = [
    'Merida', 'Cancun', 'Durango', 'Hermosillo', 'Irapuato',
    'Mazatlan', 'Culiacan', 'Chihuahua', 'Saltillo', 'Celaya',
    'Pachuca', 'Campeche', 'Reynosa', 'Obregon', 'Tecoman',
    'Tulancingo', 'Parral', 'Delicias', 'Tepic', 'Apodaca'
]

# Generar DataFrame de Clientes
clientes_df = pd.DataFrame({
    'id_cliente': clientes_unicos,
    'ciudad': [random.choice(ciudades) for _ in range(len(clientes_unicos))],
    'genero': [random.choice(['masculino', 'femenino']) for _ in range(len(clientes_unicos))]
})

# Agregar la nueva hoja al diccionario
excel['Clientes'] = clientes_df

# Guardar el archivo con la hoja nueva
with pd.ExcelWriter(archivo, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    for hoja, df in excel.items():
        df.to_excel(writer, sheet_name=hoja, index=False)

In [7]:
import pandas as pd

# Leer solo la hoja 'Clientes'
archivo = 'ventas_olap.xlsx'
clientes_df = pd.read_excel(archivo, sheet_name='Clientes')

# Buscar duplicados en la columna 'id_cliente'
duplicados = clientes_df[clientes_df.duplicated('id_cliente', keep=False)]

# Mostrar resultados
if not duplicados.empty:
    print("Se encontraron registros duplicados en 'id_cliente':")
    print(duplicados)
else:
    print("No hay registros duplicados en 'id_cliente'.")


No hay registros duplicados en 'id_cliente'.


In [ ]:
import pandas as pd

# Ruta del archivo Excel
archivo_excel = 'ventas_olap.xlsx'

# Leer las hojas del Excel
ventas_df = pd.read_excel(archivo_excel, sheet_name='Ventas')
productos_df = pd.read_excel(archivo_excel, sheet_name='Productos')
fechas_df = pd.read_excel(archivo_excel, sheet_name='Fechas')
clientes_df = pd.read_excel(archivo_excel, sheet_name='Clientes')

# Unir Ventas con Productos (por id_producto)
df_completo = pd.merge(ventas_df, productos_df, on='id_producto', how='left')

# Unir con Fechas (por id_venta e id_cliente)
df_completo = pd.merge(df_completo, fechas_df, on=['id_venta', 'id_cliente'], how='left')

# Unir con Clientes (por id_cliente)
df_completo = pd.merge(df_completo, clientes_df, on='id_cliente', how='left')

# Guardar como CSV sin modificar el Excel original
df_completo.to_csv('ventas.csv', index=False, encoding='utf-8')